<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#数据归一化" data-toc-modified-id="数据归一化-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>数据归一化</a></span></li><li><span><a href="#批归一化实现" data-toc-modified-id="批归一化实现-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>批归一化实现</a></span></li><li><span><a href="#回调函数" data-toc-modified-id="回调函数-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>回调函数</a></span></li></ul></div>

In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

In [19]:
fashion_mnist = keras.datasets.fashion_mnist
(x_train_all, y_train_all), (x_test, y_test) = fashion_mnist.load_data()
x_valid, x_train = x_train_all[:5000], x_train_all[5000:]
y_valid, y_train = y_train_all[:5000], y_train_all[5000:]

print(x_valid.shape, y_valid.shape)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(5000, 28, 28) (5000,)
(55000, 28, 28) (55000,)
(10000, 28, 28) (10000,)


### 数据归一化

In [20]:
#x = (x - μ) / std(方差) 符合均值是零，方差是1的正态分布
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
#x_train: [None, 28, 28] -> [None, 784]
x_train_scaled = scaler.fit_transform(
    x_train.astype(np.float32).reshape(-1,1)).reshape(-1,28,28)
x_valid_scaled = scaler.transform(
    x_valid.astype(np.float32).reshape(-1,1)).reshape(-1,28,28)
x_test_scaled = scaler.transform(
    x_test.astype(np.float32).reshape(-1,1)).reshape(-1,28,28)

### 使用dropout防拟合

In [21]:
#tf.keras.models.Sequential()

model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28, 28]))
for _ in range(20):
    model.add(keras.layers.Dense(100, activation="selu"))
#对前面一层进行dropout
#AlphaDropout 1.均值和方差不变 2.归一化性质也不变
model.add(keras.layers.AlphaDropout(rate=0.5))#rate丢掉的单元的数目比例
#model.add(keras.layers.Dropout(rate=0.5))

model.add(keras.layers.Dense(10,activation="softmax"))

#relu: y = max（0，x)
#softmax：将向量变成概率分布 x = [x1, x2, x3]
#    y = [e^x1/sum, e^x2/sum, e^x3/sum], sum = e^X1 + e^x2 + e^x3
#reason for sparse: y->index. y->one)hot->[]
model.compile(loss = "sparse_categorical_crossentropy",
              optimizer = "sgd",
              metrics = ["accuracy"])

In [23]:
#model.layers#查看有多少层

In [24]:
model.summary()#查看模型的架构

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_2 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_42 (Dense)             (None, 100)               78500     
_________________________________________________________________
dense_43 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_44 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_45 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_46 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_47 (Dense)             (None, 100)              

 第一层[none,784] *w(矩阵)+b（偏置） -> [none,300] w.shape[784,300],b=[300]

### 回调函数

In [25]:
#Tensorborad， earlystopping， modelcheckpoint
logdir = './dnn-selu-dropout-callbacks/'
if not os.path.exists(logdir):
    os.mkdir(logdir)
output_model_file = os.path.join(logdir,"fashion_mnist_model.h5")

callbacks = [
    keras.callbacks.TensorBoard(logdir),
    keras.callbacks.ModelCheckpoint(output_model_file,save_best_only=True),
    keras.callbacks.EarlyStopping(patience=5,min_delta=1e-3),
]
history = model.fit(x_train_scaled, y_train, epochs=10, 
                    validation_data=(x_valid_scaled, y_valid),
                   callbacks=callbacks)

Train on 55000 samples, validate on 5000 samples
Epoch 1/10
   32/55000 [..............................] - ETA: 37:36 - loss: 3.3216 - accuracy: 0.1250

ProfilerNotRunningError: Cannot stop profiling. No profiler is running.

In [10]:
def plot_learning_curves(history):
    pd.DataFrame(history.history).plot(figsize=(8,5))
    plt.grid(True)
    plt.gca().set_ylim(0,2)
    plt.show()
    
plot_learning_curves(history)

#1.参数多，训练不充分，初期目标函数不发生明显变化
#2.梯度消失 ->链式法则 ->复合函数f(g(x))
#3.selu缓解梯度消失

NameError: name 'history' is not defined

In [ ]:
#指标评估
model.evaluate(x_test_scaled, y_test)